# Grid-search results for GPs with RNNs

In [1]:
import pickle
import glob
import os
import pandas
import numpy as np
import json

In [2]:
with open('./datasets.json', 'r') as f:
    datasets = json.load(f).keys()
models = ['GRU', 'LSTM', 'SigGRU', 'SigLSTM']
column_order = ['H8_D1', 'H8_D0', 'H32_D1', 'H32_D0', 'H128_D1', 'H128_D0']

In [3]:
results_folder_template = './gridsearch/GP{}'
results_dict = {m : {'acc' : {}, 'nlpp' : {}} for m in models}

### Parse results

In [4]:
for j, model in enumerate(models):
    results_folder = results_folder_template.format(model)
    for i, dataset in enumerate(datasets):
        print('Parsing results for model: {} ({}/{}), dataset: {} ({:2d}/{})...'.format(model, j+1, len(models), dataset, i+1, len(datasets)))
#         rint('Parsing results for model: ({}/{}) dataset: ({:2d}/{})...'.format(j+1, len(models), i+1, len(datasets)))
        for num_hidden in [8, 32, 128]:
            for use_dropout in [0, 1]:
                experiment_name = '{}_H{}_D{}'.format(dataset, num_hidden, use_dropout)
                file_path = os.path.join(results_folder, experiment_name + '.pkl')

                if os.path.exists(file_path):
                    with open(file_path, 'rb') as f:
                        saved = pickle.load(f)
                    test_acc = saved['results']['test_acc']
                    test_nlpp = saved['results']['test_nlpp']
                else:
                    print('Warning: result file missing {}'.format(file_path))
                    test_acc = float('nan')
                    test_nlpp = float('nan')

                key = 'H{}_D{}'.format(num_hidden, use_dropout)
                if key not in results_dict[model]['acc']:
                    results_dict[model]['acc'][key] = []
                results_dict[model]['acc'][key].append(test_acc)

                if key not in results_dict[model]['nlpp']:
                    results_dict[model]['nlpp'][key] = []
                results_dict[model]['nlpp'][key].append(test_nlpp)

Parsing results for model: GRU (1/4), dataset: ArabicDigits ( 1/16)...
Parsing results for model: GRU (1/4), dataset: AUSLAN ( 2/16)...
Parsing results for model: GRU (1/4), dataset: CharacterTrajectories ( 3/16)...
Parsing results for model: GRU (1/4), dataset: CMUsubject16 ( 4/16)...
Parsing results for model: GRU (1/4), dataset: DigitShapes ( 5/16)...
Parsing results for model: GRU (1/4), dataset: ECG ( 6/16)...
Parsing results for model: GRU (1/4), dataset: JapaneseVowels ( 7/16)...
Parsing results for model: GRU (1/4), dataset: KickvsPunch ( 8/16)...
Parsing results for model: GRU (1/4), dataset: LIBRAS ( 9/16)...
Parsing results for model: GRU (1/4), dataset: NetFlow (10/16)...
Parsing results for model: GRU (1/4), dataset: PEMS (11/16)...
Parsing results for model: GRU (1/4), dataset: PenDigits (12/16)...
Parsing results for model: GRU (1/4), dataset: Shapes (13/16)...
Parsing results for model: GRU (1/4), dataset: UWave (14/16)...
Parsing results for model: GRU (1/4), dataset: 

### Results of grid-search over 6 architectures

In [5]:
tol = 0.1
def highlight_best_nlpp(s):
    min_s = s.min()
    l = ['' for v in s]
    for i in range(len(s)):
        if np.abs(s[i] - min_s) / min_s <= tol:
            l[i] = 'font-weight: bold'
            break
    return l

In [6]:
architectures = {model : {} for model in models}

In [7]:
from IPython.display import display
for model in models:
    
    model_full = '# GP{} #'.format(model)
    print()
    print('#'*len(model_full))
    print(model_full)
    print('#'*len(model_full))
    print()
    
    print('Accuracy:'.format(model))
    df = pandas.DataFrame.from_dict(results_dict[model]['acc']).rename(index={i : name for i, name in enumerate(datasets)})
    df = df[column_order]
    display(df)
    
    print()
    
    print('Negative log-predictive probability:'.format(model))
    df = pandas.DataFrame.from_dict(results_dict[model]['nlpp']).rename(index={i : name for i, name in enumerate(datasets)})
    df = df[column_order]
    # get best architecture    
    for row in df.iterrows():
        dataset = row[0]
        results = row[1]
        labels = results.index
        nlpps = list(results)
        min_nlpp = np.min(nlpps)
        for i in range(len(nlpps)):
            if np.abs(nlpps[i] - min_nlpp) / min_nlpp <= tol:
                keys = labels[i].split('_')
                architectures[model][dataset] = {x[0] : int(x[1:]) for x in keys}
                break
                
    df = df.style.apply(highlight_best_nlpp, axis=1)
    display(df)


#########
# GPGRU #
#########

Accuracy:


,H8_D1,H8_D0,H32_D1,H32_D0,H128_D1,H128_D0
ArabicDigits,0.826515,0.949242,0.987879,0.987879,0.996970,0.991667
AUSLAN,0.000000,0.026316,0.000000,0.885965,0.008772,0.973684
CharacterTrajectories,0.050000,0.050000,0.016667,0.283333,0.050000,0.050000
CMUsubject16,0.833333,1.000000,1.000000,1.000000,1.000000,0.833333
DigitShapes,1.000000,0.750000,1.000000,0.875000,0.625000,0.875000
ECG,0.700000,0.900000,0.800000,0.800000,0.850000,0.850000
JapaneseVowels,0.962963,0.925926,0.962963,0.981481,0.944444,1.000000
KickvsPunch,0.500000,0.500000,0.750000,0.250000,0.250000,0.500000
LIBRAS,0.277778,0.500000,0.472222,0.638889,0.500000,0.722222
NetFlow,0.863354,0.956522,0.913043,0.913043,0.888199,0.919255



Negative log-predictive probability:


,H8_D1,H8_D0,H32_D1,H32_D0,H128_D1,H128_D0
ArabicDigits,0.927026,0.267272,0.0490473,0.0549373,0.0257298,0.0313868
AUSLAN,5.35624,6.72953,6.09275,0.559439,7.37102,0.17324
CharacterTrajectories,3.09567,3.0696,3.93735,4.41635,2.99901,2.99901
CMUsubject16,0.171565,0.0552333,0.0399411,0.0419835,0.0681892,0.48349
DigitShapes,0.0713792,0.68017,0.123715,0.497922,0.929243,0.262527
ECG,0.579443,0.412381,0.541743,0.490925,0.481383,0.360336
JapaneseVowels,0.162194,0.245833,0.0938614,0.127716,0.219717,0.0507244
KickvsPunch,0.566845,0.793468,0.682104,1.60007,0.976741,0.644622
LIBRAS,4.2794,2.71671,2.55397,1.86423,1.73887,1.13792
NetFlow,0.367539,0.152262,0.283224,0.191108,0.319773,0.22604



##########
# GPLSTM #
##########

Accuracy:


...

,H8_D1,H8_D0,H32_D1,H32_D0,H128_D1,H128_D0
ArabicDigits,0.0088006,0.0136365,0.00509234,0.0129831,0.00653097,0.00917928
AUSLAN,1.073,0.112016,0.335605,0.082188,0.0641414,0.0544736
CharacterTrajectories,0.482527,0.0443241,0.03097,0.0471635,0.023593,0.0762512
CMUsubject16,0.231363,0.184248,0.0370257,0.266088,0.226665,0.140845
DigitShapes,0.0400083,0.185914,0.0297911,0.0556762,0.0232964,0.115417
ECG,0.432611,0.271009,0.506099,0.317543,0.393618,0.182979
JapaneseVowels,0.0353596,0.0966153,0.213887,0.0996828,0.00990296,0.0673548
KickvsPunch,0.707892,0.777597,0.625231,0.594271,0.30245,0.46099
LIBRAS,1.04145,0.843198,0.826755,0.222336,1.13082,1.04888
NetFlow,0.290612,0.253032,0.305505,0.186136,0.184877,0.29429



#############
# GPSigLSTM #
#############

Accuracy:


,H8_D1,H8_D0,H32_D1,H32_D0,H128_D1,H128_D0
ArabicDigits,0.996212,0.996212,0.996970,0.995455,0.997727,0.999242
AUSLAN,0.947368,0.956140,0.978070,0.991228,0.973684,0.982456
CharacterTrajectories,0.933333,1.000000,0.933333,0.983333,1.000000,1.000000
CMUsubject16,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
DigitShapes,1.000000,1.000000,1.000000,1.000000,1.000000,0.750000
ECG,0.900000,0.800000,0.800000,0.850000,0.800000,0.800000
JapaneseVowels,0.981481,0.962963,0.962963,0.962963,0.962963,0.981481
KickvsPunch,0.750000,1.000000,0.250000,0.750000,1.000000,0.250000
LIBRAS,0.805556,0.750000,0.722222,0.861111,0.805556,0.861111
NetFlow,0.950311,0.956522,0.937888,0.950311,0.906832,0.956522



Negative log-predictive probability:


,H8_D1,H8_D0,H32_D1,H32_D0,H128_D1,H128_D0
ArabicDigits,0.0132089,0.0141441,0.00913112,0.0202521,0.00817644,0.0103126
AUSLAN,0.401072,0.154767,0.133415,0.0760032,0.128882,0.127232
CharacterTrajectories,0.323259,0.0325493,0.401811,0.0496477,0.0185265,0.0576778
CMUsubject16,0.166285,0.184627,0.0214178,0.175328,0.134255,0.210017
DigitShapes,0.0435722,0.329822,0.0263275,0.0768114,0.0349058,0.367786
ECG,0.36951,0.435747,0.48699,0.420502,0.463573,0.428916
JapaneseVowels,0.106946,0.105852,0.133065,0.0970364,0.0911406,0.0398849
KickvsPunch,0.469701,0.5242,0.892543,0.546145,0.242483,1.10033
LIBRAS,1.42276,1.31663,1.34326,0.65155,0.918592,0.682695
NetFlow,0.139279,0.145993,0.211677,0.173035,0.213894,0.147137


In [8]:
pandas.DataFrame.from_dict({m : ['H={}, D={}'.format(architectures[m][d]['H'], architectures[m][d]['D']) for d in datasets] for m in models}).rename(index={i : name for i, name in enumerate(datasets)})

,GRU,LSTM,SigGRU,SigLSTM
ArabicDigits,"H=128, D=1","H=128, D=1","H=32, D=1","H=128, D=1"
AUSLAN,"H=128, D=0","H=128, D=0","H=128, D=0","H=32, D=0"
CharacterTrajectories,"H=8, D=1","H=128, D=1","H=128, D=1","H=128, D=1"
CMUsubject16,"H=32, D=1","H=32, D=1","H=32, D=1","H=32, D=1"
DigitShapes,"H=8, D=1","H=128, D=1","H=128, D=1","H=32, D=1"
ECG,"H=128, D=0","H=128, D=1","H=128, D=0","H=8, D=1"
JapaneseVowels,"H=128, D=0","H=128, D=1","H=128, D=1","H=128, D=0"
KickvsPunch,"H=8, D=1","H=8, D=0","H=128, D=1","H=128, D=1"
LIBRAS,"H=128, D=0","H=128, D=0","H=32, D=0","H=32, D=0"
NetFlow,"H=8, D=0","H=32, D=0","H=32, D=0","H=8, D=1"


In [9]:
with open('./architectures.json', 'w') as f:
    json.dump(architectures, f)
architectures

{'GRU': {'ArabicDigits': {'H': 128, 'D': 1},
  'AUSLAN': {'H': 128, 'D': 0},
  'CharacterTrajectories': {'H': 8, 'D': 1},
  'CMUsubject16': {'H': 32, 'D': 1},
  'DigitShapes': {'H': 8, 'D': 1},
  'ECG': {'H': 128, 'D': 0},
  'JapaneseVowels': {'H': 128, 'D': 0},
  'KickvsPunch': {'H': 8, 'D': 1},
  'LIBRAS': {'H': 128, 'D': 0},
  'NetFlow': {'H': 8, 'D': 0},
  'PEMS': {'H': 32, 'D': 0},
  'PenDigits': {'H': 128, 'D': 0},
  'Shapes': {'H': 8, 'D': 1},
  'UWave': {'H': 32, 'D': 1},
  'Wafer': {'H': 128, 'D': 0},
  'WalkvsRun': {'H': 128, 'D': 1}},
 'LSTM': {'ArabicDigits': {'H': 128, 'D': 1},
  'AUSLAN': {'H': 128, 'D': 0},
  'CharacterTrajectories': {'H': 128, 'D': 1},
  'CMUsubject16': {'H': 32, 'D': 1},
  'DigitShapes': {'H': 128, 'D': 1},
  'ECG': {'H': 128, 'D': 1},
  'JapaneseVowels': {'H': 128, 'D': 1},
  'KickvsPunch': {'H': 8, 'D': 0},
  'LIBRAS': {'H': 128, 'D': 0},
  'NetFlow': {'H': 32, 'D': 0},
  'PEMS': {'H': 8, 'D': 1},
  'PenDigits': {'H': 128, 'D': 1},
  'Shapes': {'H': 